In [ ]:
import pandas as pd

# Input and output file paths
input_path = r'C:\Users\51195\Desktop\表格.xlsx'
output_path = r'C:\Users\51195\Desktop\表格_full_chains.xlsx'

# Read Excel, assuming the first row is the header
df = pd.read_excel(input_path)

# Automatically get the column names of columns A and B (assuming they are columns 1 and 2)
colA, colB = df.columns[0], df.columns[1]

# Construct a mapping from A → B
mapping = dict(zip(df[colA], df[colB]))

# Helper: Generate Excel column names (1→A, 2→B, …, 27→AA)
def excel_col_name(n):
    name = ''
    while n > 0:
        n, rem = divmod(n-1, 26)
        name = chr(65 + rem) + name
    return name

# Extract chain relationships (elements appear at most twice), and track at most max_depth steps to avoid excessive length
def extract_chain_limit_dups(start_value, mapping, max_depth):
    chain = []
    counts = {}
    current = mapping.get(start_value)
    steps = 0
    while current is not None and steps < max_depth:
        # If the current element has appeared twice, stop
        if counts.get(current, 0) >= 2:
            break
        # Otherwise join the chain and update the number of occurrences
        chain.append(current)
        counts[current] = counts.get(current, 0) + 1
        # Continue to chase down
        current = mapping.get(current)
        steps += 1
    return chain

# To be safe, the maximum depth is set to twice the size of the map.
max_depth = 2 * len(mapping)

# For each row, start the chain from the value in column B, allowing repetitions but no more than twice
chains = df[colB].apply(lambda x: extract_chain_limit_dups(x, mapping, max_depth)).tolist()

# Determine the length of the longest chain to generate the corresponding number of columns (C, D, E…)
max_len = max((len(c) for c in chains), default=0)
extra_cols = [excel_col_name(i) for i in range(3, 3 + max_len)]

# Expand all chains into a new DataFrame and concatenate them with the original table
chains_df = pd.DataFrame(chains, columns=extra_cols)
result_df = pd.concat([df, chains_df], axis=1)

# Save the complete table with columns C, D, E…
result_df.to_excel(output_path, index=False)

print(f'已导出含链式关系（每个元素最多出现两次）的完整表格到：{output_path}')
